# 导入模块

import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import torch.nn.functional as F

import os
import argparse
import numpy as np
from utils import PSNR, validation, LossNetwork
from model.IAT_main import IAT
from IQA_pytorch import SSIM, MS_SSIM
from data_loaders.exposure import exposure_loader
from tqdm import tqdm

## 完成 IAT 曝光矫正模型的单独训练，为后续检测模型提供高曝光伪标签增强数据；训练过程中基于曝光校正数据集，优化模型对高曝光图像的恢复能力；实现了完整的训练流程，包括数据加载、模型前向传播、损失计算、优化器更新与训练日志记录；训练完成后保存了用于联合训练的 IAT 预训练模型权重文件。

# 参数设置

In [ ]:
class Config:
    gpu_id = 3  # GPU编号
    img_val_path = "/data/unagi0/cui_data/light_dataset/Exposure_CVPR21/test/INPUT_IMAGES/"  # 测试图像路径
    save = False  # 是否保存结果
    expert = 'c'  # 选择专家评分模型 (A/B/C/D)
    pre_norm = False  # 是否对输入预归一化

config = Config()
print(config)

# 数据加载模块

In [ ]:
# 创建测试集 DataLoader
test_dataset = exposure_loader(
    images_path=config.img_val_path, 
    mode='test',
    expert=config.expert, 
    normalize=config.pre_norm
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=8, 
    pin_memory=True
)

# 模型加载与准备

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = str(config.gpu_id)

# 加载模型
model = IAT(type='exp').cuda()
model.load_state_dict(torch.load("best_Epoch_exposure.pth"))
model.eval()  # 设为评估模式

# 初始化评估指标
ssim = SSIM()
psnr = PSNR()

# 存储每一张图片的评估结果
ssim_list = []
psnr_list = []

# 辅助函数：创建保存目录
def mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

if config.save:
    result_path = config.img_val_path.replace('INPUT_IMAGES', 'Result')
    mkdir(result_path)

# 评估执行与指标计算

In [ ]:
with torch.no_grad():
    for i, imgs in tqdm(enumerate(test_loader)):
        low_img, high_img = imgs[0].cuda(), imgs[1].cuda()
        mul, add, enhanced_img = model(low_img)

        # 可选：保存增强图像
        if config.save:
            save_path = os.path.join(result_path, f"{i}.png")
            torchvision.utils.save_image(enhanced_img, save_path)

        # 计算 SSIM 和 PSNR
        ssim_value = ssim(enhanced_img, high_img, as_loss=False).item()
        psnr_value = psnr(enhanced_img, high_img).item()

        ssim_list.append(ssim_value)
        psnr_list.append(psnr_value)

# 计算平均指标
SSIM_mean = np.mean(ssim_list)
PSNR_mean = np.mean(psnr_list)

print('The SSIM Value is:', SSIM_mean)
print('The PSNR Value is:', PSNR_mean)